In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.document_loaders import PyPDFLoader
loaders = [
    PyPDFLoader("LectureNotes/cs229-notes1.pdf"),
    PyPDFLoader("LectureNotes/cs229-notes1.pdf"),
    PyPDFLoader("LectureNotes/cs229-notes2.pdf"),
    PyPDFLoader("LectureNotes/cs229-notes3.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [4]:
from langchain.text_splitter import  RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)

In [5]:
len(splits)

163

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

C:\Users\busra\AppData\Local\Temp\ipykernel_8156\3400180925.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [8]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [9]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [10]:
len(embedding1)


1536

In [11]:
import numpy as np
np.dot(embedding1, embedding2)

0.9631510802407719

In [12]:
np.dot(embedding1, embedding3)

0.7702031204123156

In [13]:
np.dot(embedding2, embedding3)

0.7590539714454778

In [15]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'
vectordb = Chroma.from_documents(
    documents = splits,
    embedding = embedding,
    persist_directory = persist_directory
)

In [16]:
print(vectordb._collection.count())

326


In [17]:
question = 'Is there an email i can ask for help'

In [18]:
docs = vectordb.similarity_search(question, k=3)

In [19]:
len(docs)

3

In [20]:
docs[0].page_content

'diﬀerent documents. For instance, if an email starts with “A NIPS . . . ,”\nthen x1 = 1 (“a” is the ﬁrst word in the dictionary), and x2 = 35000 (if\n“nips” is the 35000th word in the dictionary).\nIn the multinomial event model, we assume that the way an email is\ngenerated is via a random process in which spam/non-spam is ﬁrst de ter-\nmined (according to p(y)) as before. Then, the sender of the email writes the\nemail by ﬁrst generating x1 from some multinomial distribution over words\n(p(x1|y)). Next, the second word x2 is chosen independently of x1 but from\nthe same multinomial distribution, and similarly for x3, x4, and so on, until\nall n words of the email have been generated. Thus, the overall proba bility of\na message is given by p(y) ∏ n\ni=1 p(xi|y). Note that this formula looks like the\none we had earlier for the probability of a message under the multi-va riate\nBernoulli event model, but that the terms in the formula now mean ve ry dif-\nferent things. In particular 

In [21]:
question = "what did they say about matlab?"

In [22]:
docs = vectordb.similarity_search(question,k=5)

In [23]:
docs[0]

Document(metadata={'page': 14, 'source': 'LectureNotes/cs229-notes1.pdf'}, page_content='that we saw earlier is known as a parametric learning algorithm, because\nit has a ﬁxed, ﬁnite number of parameters (the θi’s), which are ﬁt to the\ndata. Once we’ve ﬁt the θi’s and stored them away, we no longer need to\nkeep the training data around to make future predictions. In cont rast, to\nmake predictions using locally weighted linear regression, we need to k eep\nthe entire training set around. The term “non-parametric” (roug hly) refers\nto the fact that the amount of stuﬀ we need to keep in order to rep resent the\nhypothesis h grows linearly with the size of the training set.\n4If x is vector-valued, this is generalized to be w(i) = exp( −(x(i) − x)T (x(i) − x)/ (2τ2)),\nor w(i) = exp( −(x(i) − x)T Σ −1(x(i) − x)/ 2), for an appropriate choice of τ or Σ.')

In [24]:
question = "what did they say about regression in the third lecture?"

In [25]:
docs = vectordb.similarity_search(question,k=5)

In [26]:
for doc in docs:
    print(doc.metadata)

{'page': 2, 'source': 'LectureNotes/cs229-notes1.pdf'}
{'page': 2, 'source': 'LectureNotes/cs229-notes1.pdf'}
{'page': 2, 'source': 'LectureNotes/cs229-notes1.pdf'}
{'page': 2, 'source': 'LectureNotes/cs229-notes1.pdf'}
{'page': 18, 'source': 'LectureNotes/cs229-notes1.pdf'}


In [27]:
print(docs[4].page_content)

In the 1960s, this “perceptron” was argued to be a rough model f or how
individual neurons in the brain work. Given how simple the algorithm is, it
will also provide a starting point for our analysis when we talk about learning
theory later in this class. Note however that even though the perc eptron may
be cosmetically similar to the other algorithms we talked about, it is act ually
a very diﬀerent type of algorithm than logistic regression and least s quares
linear regression; in particular, it is diﬃcult to endow the perceptron ’s predic-
tions with meaningful probabilistic interpretations, or derive the pe rceptron
as a maximum likelihood estimation algorithm.


In [28]:
print(docs[2].page_content)

3
Part I
Linear Regression
To make our housing example more interesting, let’s consider a slightlyricher
dataset in which we also know the number of bedrooms in each house:
Living area (feet 2)
#bedrooms Price (1000$s)
2104 3 400
1600 3 330
2400 3 369
1416 2 232
3000 4 540
.
.
.
.
.
.
.
.
.
Here, the x’s are two-dimensional vectors in R2. For instance, x(i)
1 is the
living area of the i-th house in the training set, and x(i)
2 is its number of
bedrooms. (In general, when designing a learning problem, it will be up to
you to decide what features to choose, so if you are out in Portland gathering
housing data, you might also decide to include other features such a s whether
each house has a ﬁreplace, the number of bathrooms, and so on. W e’ll say
more about feature selection later, but for now let’s take the feat ures as
given.)
To perform supervised learning, we must decide how we’re going to re p-
resent functions/hypotheses h in a computer. As an initial choice, let’s say
we decide to